In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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'
    
)

response=llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
pip install langchain_community beautifulsoup4

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.5 MB 4.1 MB/s eta 0:00:01
   ------- -------------------------------- 0.5/2.5 MB 5.2 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.5 MB 6.8 MB/s eta 0:00:01
   ---------------------- ----------------- 1.4/2.5 MB 8.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 9.0 MB/s eta 0:00:01
   ------------------------------------- -- 2.3/2.5 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
   ---------------------------------------- 50.9/50.9 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/senior-manager-ai-ml-engineering/job/R-46903")
page_data=loader.load().pop().page_content
print(page_data)






















Senior Manager - AI/ML Engineering










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Sel

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page data}
    ###INSTRUCTION:
    The scraped text is from the career's page of website.
    Your job is to extract the job postings and return them in JSON format containing
    following keys:'role','experience','skills' and 'description'.
    only return the valid JSON.
    ### VALID JSON(No PREAMBLE):
    """
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page data':page_data})
type(res.content)

str

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Senior Manager - AI/ML Engineering',
 'experience': '7+ years of experience across multiple roles in developing productionized code in software or data engineering, machine learning or a related field',
 'skills': ['Machine learning',
  'Artificial intelligence',
  'Data science',
  'Software engineering',
  'Cloud computing',
  'ETL',
  'ML',
  'Analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, Databricks, AWS'],
 'description': 'Nike is looking for an experienced Engineering leader who can lead and grow teams of machine learning engineers, data scientists, and software architects to deliver scalable machine learning and artificial intelligence solutions to customers across our business.'}

In [17]:
type(json_res)

dict

In [18]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [22]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [26]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [25]:
job=json_res
job['skills']

['Machine learning',
 'Artificial intelligence',
 'Data science',
 'Software engineering',
 'Cloud computing',
 'ETL',
 'ML',
 'Analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, Databricks, AWS']

In [28]:
prompt_email=PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Megala, a business development executive at Visha.Visha is an AI & Software Consulting company dedicated to face
    the seamless integration of business processess through automated tools.
    Over our experience,we have empowered numerous enterprises with tailored solutions,fostering scalability,
    process optimization,cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Visha's portfolio: {link_list}
    Remember you are Megala,BDE at Visha.
    Do not provide a preamble
    ### EMAIL(NO PREAMBLE)
    """
    )
chain_email=prompt_email | llm
res=chain_email.invoke({"job_description":str(job), "link_list": links})
print(res.content)

Subject: Expert AI/ML Engineering Solutions for Nike's Business Growth

Dear Hiring Manager,

I came across the job description for a Senior Manager - AI/ML Engineering at Nike, and I'm excited to introduce Visha, an AI & Software Consulting company that can help fulfill your requirements. With our expertise in developing productionized code in software or data engineering, machine learning, and related fields, we're confident in our ability to deliver scalable machine learning and artificial intelligence solutions to drive business growth.

Our team of experienced machine learning engineers, data scientists, and software architects can help lead and grow your teams to achieve exceptional results. We have a strong background in machine learning, artificial intelligence, data science, software engineering, cloud computing, ETL, and analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, Databricks, and AWS.

To demonstrate our capabilities, I'd like to share